In [1]:
import ast
import pandas as pd
import numpy as np

## 2020 - 2024 데이터 크롤링 후 nan처리 ASOS_data_preprocessed_V1

In [2]:
path = '/Volumes/ESD-ISO/project_preson/project_data/웹크롤링데이터/2020-2024ASOS_관측데이터.csv'
asos_df = pd.read_csv(path)
asos_df = asos_df.iloc[:, 0].apply(ast.literal_eval)  # 문자열을 딕셔너리로 변환
asos_df = pd.json_normalize(asos_df)  # 딕셔너리를 열로

In [3]:
asos_df

,tm,hm,pv,ps,td,pa,dc10Tca,clfmAbbrCd,lcsCh,m005Te,m01Te,m02Te,m03Te,vs,ws,wd,rn
0,2020-01-01 01:00,40,1.6,1033.0,-17.3,1021.8,9,Sc,7,-0.8,0.7,2.1,3.2,2000,1.7,50,
1,2020-01-01 02:00,42,1.7,1032.8,-16.5,1021.6,9,Sc,7,-0.8,0.6,2.0,3.1,1988,0.1,0,
2,2020-01-01 03:00,46,1.8,1032.7,-15.4,1021.5,9,Sc,14,-0.8,0.6,2.0,3.1,2000,0.0,0,0.0
3,2020-01-01 04:00,50,2.0,1032.8,-14.2,1021.6,8,Sc,6,-0.8,0.6,2.0,3.0,1908,0.0,0,
4,2020-01-01 05:00,55,2.3,1032.3,-12.8,1021.1,9,Sc,5,-0.7,0.6,2.0,3.0,1999,0.0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43648,2024-12-30 21:00,67,6.1,1016.0,0.0,1005.4,6,Ci,,-0.1,-0.3,0.8,1.7,1053,2.7,270,
43649,2024-12-30 22:00,71,6.1,1016.2,0.0,1005.6,0,,,-0.1,-0.3,0.8,1.7,906,2.4,270,
43650,2024-12-30 23:00,75,6.1,1016.0,0.0,1005.4,0,,,-0.1,-0.3,0.8,1.7,754,0.7,250,
43651,2024-12-31 00:00,81,7.0,1015.8,1.8,1005.2,9,,,-0.1,-0.3,0.8,1.7,563,2.3,250,0.0


In [4]:
asos_df['rn'] = asos_df['rn'].map(lambda x : float(x.split()[0]) if x else 0.0) # 비가 안왔을 경우 NaN -> 강수량을 0.0으로 변환

In [5]:
# 딕셔너리 문자열이 원본 데이터였기에 type을 바꿔줄 필요가 있음
# 변환할 열 리스트
columns_to_convert = ['hm', 'pv', 'ps', 'td', 'pa', 'dc10Tca', 'lcsCh', 'm005Te', 'm01Te', 'm02Te', 'm03Te', 'vs', 'ws', 'wd']

# 열 단위로 값 변환
asos_df[columns_to_convert] = asos_df[columns_to_convert].applymap(lambda x: float(x) if x != '' else np.nan)

In [6]:
asos_df['clfmAbbrCd'] = asos_df['clfmAbbrCd'].map(lambda x : np.nan if x == '' else x)

In [7]:
asos_df['rainy_day'] = (asos_df['rn'] > 0.0).astype(dtype=int) # 강수량이 0보다 클 경우 1, 0일경우 0

In [8]:
asos_df[asos_df['clfmAbbrCd'].isna()] # lcsCh = 최저운고값이 있는데 운형이 없는경우 시야가 흐릴 가능성이 있음

,tm,hm,pv,ps,td,pa,dc10Tca,clfmAbbrCd,lcsCh,m005Te,m01Te,m02Te,m03Te,vs,ws,wd,rn,rainy_day
11,2020-01-01 22:00,71.0,4.1,1031.8,-5.5,1020.8,8.0,NaN,12.0,-0.1,0.6,1.7,2.7,897.0,1.5,50.0,0.0,0
12,2020-01-01 23:00,70.0,4.0,1032.0,-5.6,1021.0,9.0,NaN,12.0,-0.1,0.5,1.7,2.7,895.0,1.0,50.0,0.0,0
13,2020-01-02 00:00,70.0,4.1,1031.3,-5.5,1020.3,9.0,NaN,NaN,-0.1,0.6,1.7,2.7,897.0,2.1,70.0,0.0,0
14,2020-01-02 01:00,72.0,4.2,1030.8,-5.1,1019.8,8.0,NaN,10.0,-0.1,0.6,1.7,2.6,857.0,2.2,50.0,0.0,0
15,2020-01-02 02:00,70.0,4.1,1030.4,-5.4,1019.4,9.0,NaN,10.0,-0.1,0.6,1.7,2.7,887.0,2.2,70.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43647,2024-12-30 20:00,68.0,6.4,1015.8,0.6,1005.3,6.0,NaN,73.0,-0.1,-0.3,0.8,1.7,1018.0,2.6,250.0,0.0,0
43649,2024-12-30 22:00,71.0,6.1,1016.2,0.0,1005.6,0.0,NaN,NaN,-0.1,-0.3,0.8,1.7,906.0,2.4,270.0,0.0,0
43650,2024-12-30 23:00,75.0,6.1,1016.0,0.0,1005.4,0.0,NaN,NaN,-0.1,-0.3,0.8,1.7,754.0,0.7,250.0,0.0,0
43651,2024-12-31 00:00,81.0,7.0,1015.8,1.8,1005.2,9.0,NaN,NaN,-0.1,-0.3,0.8,1.7,563.0,2.3,250.0,0.0,0


In [9]:
def no_clouds(row):
    if (pd.isna(row['lcsCh'])) & (pd.isna(row['clfmAbbrCd'])):
        row['clfmAbbrCd'] = 'no_clouds'
        return row
    else:
        return row

In [10]:
asos_df = asos_df.apply(lambda x : no_clouds(x), axis=1)

In [11]:
asos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43653 entries, 0 to 43652
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tm          43653 non-null  object 
 1   hm          43653 non-null  float64
 2   pv          43653 non-null  float64
 3   ps          43643 non-null  float64
 4   td          43653 non-null  float64
 5   pa          43643 non-null  float64
 6   dc10Tca     43633 non-null  float64
 7   clfmAbbrCd  39852 non-null  object 
 8   lcsCh       22641 non-null  float64
 9   m005Te      43639 non-null  float64
 10  m01Te       43639 non-null  float64
 11  m02Te       43639 non-null  float64
 12  m03Te       43639 non-null  float64
 13  vs          43653 non-null  float64
 14  ws          43583 non-null  float64
 15  wd          43583 non-null  float64
 16  rn          43653 non-null  float64
 17  rainy_day   43653 non-null  int64  
dtypes: float64(15), int64(1), object(2)
memory usage: 6.0+ MB


In [12]:
asos_df[(asos_df['lcsCh'].isna()) | (asos_df['clfmAbbrCd'].isna())] # 둘중 하나만 nan인 데이터 확인

,tm,hm,pv,ps,td,pa,dc10Tca,clfmAbbrCd,lcsCh,m005Te,m01Te,m02Te,m03Te,vs,ws,wd,rn,rainy_day
11,2020-01-01 22:00,71.0,4.1,1031.8,-5.5,1020.8,8.0,NaN,12.0,-0.1,0.6,1.7,2.7,897.0,1.5,50.0,0.0,0
12,2020-01-01 23:00,70.0,4.0,1032.0,-5.6,1021.0,9.0,NaN,12.0,-0.1,0.5,1.7,2.7,895.0,1.0,50.0,0.0,0
13,2020-01-02 00:00,70.0,4.1,1031.3,-5.5,1020.3,9.0,no_clouds,NaN,-0.1,0.6,1.7,2.7,897.0,2.1,70.0,0.0,0
14,2020-01-02 01:00,72.0,4.2,1030.8,-5.1,1019.8,8.0,NaN,10.0,-0.1,0.6,1.7,2.6,857.0,2.2,50.0,0.0,0
15,2020-01-02 02:00,70.0,4.1,1030.4,-5.4,1019.4,9.0,NaN,10.0,-0.1,0.6,1.7,2.7,887.0,2.2,70.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43648,2024-12-30 21:00,67.0,6.1,1016.0,0.0,1005.4,6.0,Ci,NaN,-0.1,-0.3,0.8,1.7,1053.0,2.7,270.0,0.0,0
43649,2024-12-30 22:00,71.0,6.1,1016.2,0.0,1005.6,0.0,no_clouds,NaN,-0.1,-0.3,0.8,1.7,906.0,2.4,270.0,0.0,0
43650,2024-12-30 23:00,75.0,6.1,1016.0,0.0,1005.4,0.0,no_clouds,NaN,-0.1,-0.3,0.8,1.7,754.0,0.7,250.0,0.0,0
43651,2024-12-31 00:00,81.0,7.0,1015.8,1.8,1005.2,9.0,no_clouds,NaN,-0.1,-0.3,0.8,1.7,563.0,2.3,250.0,0.0,0


In [13]:
def no_lcsCh(row):
    if (pd.isna(row['lcsCh'])) & (row['clfmAbbrCd'] == 'no_clouds'):
        row['lcsCh'] = 0.0
        return row
    else:
        return row

In [14]:
# 구름이 없을때 최저운고를 0으로 변환하기위한 최저운고 데이터 확인
asos_df[(asos_df['lcsCh'] == 0.0)] # 0에 데이터가 없음으로 구름이 없을때 최저운고를 0으로 대치
asos_df = asos_df.apply(lambda x : no_lcsCh(x), axis=1)
asos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43653 entries, 0 to 43652
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tm          43653 non-null  object 
 1   hm          43653 non-null  float64
 2   pv          43653 non-null  float64
 3   ps          43643 non-null  float64
 4   td          43653 non-null  float64
 5   pa          43643 non-null  float64
 6   dc10Tca     43633 non-null  float64
 7   clfmAbbrCd  39852 non-null  object 
 8   lcsCh       38231 non-null  float64
 9   m005Te      43639 non-null  float64
 10  m01Te       43639 non-null  float64
 11  m02Te       43639 non-null  float64
 12  m03Te       43639 non-null  float64
 13  vs          43653 non-null  float64
 14  ws          43583 non-null  float64
 15  wd          43583 non-null  float64
 16  rn          43653 non-null  float64
 17  rainy_day   43653 non-null  int64  
dtypes: float64(15), int64(1), object(2)
memory usage: 6.0+ MB


In [15]:
# 운형이 있지만 최저운고가 없는경우 선형보간법으로 대치
asos_df['lcsCh'] = asos_df['lcsCh'].interpolate(method='linear')

In [16]:
# 최저운고가 있지만 운형이 없는경우 시간적 보간법으로 대치
asos_df['clfmAbbrCd'] = asos_df['clfmAbbrCd'].fillna(method='ffill').fillna(method='bfill')

In [17]:
asos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43653 entries, 0 to 43652
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tm          43653 non-null  object 
 1   hm          43653 non-null  float64
 2   pv          43653 non-null  float64
 3   ps          43643 non-null  float64
 4   td          43653 non-null  float64
 5   pa          43643 non-null  float64
 6   dc10Tca     43633 non-null  float64
 7   clfmAbbrCd  43653 non-null  object 
 8   lcsCh       43653 non-null  float64
 9   m005Te      43639 non-null  float64
 10  m01Te       43639 non-null  float64
 11  m02Te       43639 non-null  float64
 12  m03Te       43639 non-null  float64
 13  vs          43653 non-null  float64
 14  ws          43583 non-null  float64
 15  wd          43583 non-null  float64
 16  rn          43653 non-null  float64
 17  rainy_day   43653 non-null  int64  
dtypes: float64(15), int64(1), object(2)
memory usage: 6.0+ MB


In [18]:
asos_df.to_csv('/Volumes/ESD-ISO/project_preson/shared_data_preparation/ASOS_data_preprocessed_V1.csv', index=False)

## 데이터 추가 크롤링(1980-1996) 후 concat과 nan처리 ASOS_data_preprocessed_V2
- 1980년도부터 2020년도까지 하려 했으나 API의 데이터가 하루에 10만건으로 제한됨에 따라 나눠서 진행
- V1과 동일한 방법으로 전처리

In [19]:
asos_dataframes = {}
base_path = '/Volumes/ESD-ISO/project_preson/project_data/웹크롤링데이터/'

for year in range(1980, 1985):  # 1980년부터 1984년까지
    file_path = f"{base_path}{year}0101-{year}1231ASOS_관측데이터.csv"
    asos_dataframes[f"asos_{year}_df"] = pd.read_csv(file_path)

In [20]:
asos_1980_1984_df = pd.DataFrame([], columns=asos_dataframes['asos_1980_df'].columns)
for asos_datafram in asos_dataframes:
    asos_1980_1984_df = pd.concat([asos_1980_1984_df,asos_dataframes[asos_datafram]])
asos_1980_1984_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37592 entries, 0 to 2649
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   hm          37592 non-null  object 
 1   pv          12529 non-null  float64
 2   ps          9135 non-null   float64
 3   td          9131 non-null   float64
 4   pa          9135 non-null   float64
 5   dc10Tca     9128 non-null   float64
 6   clfmAbbrCd  9135 non-null   float64
 7   lcsCh       6271 non-null   object 
 8   m005Te      3556 non-null   float64
 9   m01Te       6265 non-null   float64
 10  m02Te       6265 non-null   float64
 11  m03Te       6265 non-null   float64
 12  vs          6265 non-null   float64
 13  ws          9135 non-null   float64
 14  wd          37582 non-null  float64
 15  rn          37592 non-null  object 
 16  tm          3531 non-null   float64
dtypes: float64(14), object(3)
memory usage: 5.2+ MB


In [21]:
asos_1980_1984_df.columns = ['tm', 'hm', 'pv', 'ps', 'td', 'pa', 'dc10Tca', 'clfmAbbrCd', 'lcsCh', 'm005Te', 'm01Te', 'm02Te', 'm03Te', 'vs', 'ws', 'wd', 'rn']

In [22]:
asos_1980_1984_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37592 entries, 0 to 2649
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tm          37592 non-null  object 
 1   hm          12529 non-null  float64
 2   pv          9135 non-null   float64
 3   ps          9131 non-null   float64
 4   td          9135 non-null   float64
 5   pa          9128 non-null   float64
 6   dc10Tca     9135 non-null   float64
 7   clfmAbbrCd  6271 non-null   object 
 8   lcsCh       3556 non-null   float64
 9   m005Te      6265 non-null   float64
 10  m01Te       6265 non-null   float64
 11  m02Te       6265 non-null   float64
 12  m03Te       6265 non-null   float64
 13  vs          9135 non-null   float64
 14  ws          37582 non-null  float64
 15  wd          37592 non-null  object 
 16  rn          3531 non-null   float64
dtypes: float64(14), object(3)
memory usage: 5.2+ MB


In [23]:
asos_dataframes = {}
base_path = '/Volumes/ESD-ISO/project_preson/project_data/웹크롤링데이터/'

for year in range(1985, 1997):  # 1985년부터 1996년까지
    file_path = f"{base_path}{year}0101-{year}1231ASOS_관측데이터.csv"
    asos_dataframes[f"asos_{year}_df"] = pd.read_csv(file_path)

In [24]:
asos_dataframes

{'asos_1985_df':                     tm    hm   pv      ps   td      pa  dc10Tca clfmAbbrCd  \
 0     1985-01-01 01:00   NaN  NaN     NaN  NaN     NaN      NaN        NaN   
 1     1985-01-01 02:00   NaN  NaN     NaN  NaN     NaN      NaN        NaN   
 2     1985-01-01 03:00  91.0  3.9  1031.0 -6.0  1019.7      3.0         Sc   
 3     1985-01-01 04:00   NaN  NaN     NaN  NaN     NaN      NaN        NaN   
 4     1985-01-01 05:00   NaN  NaN     NaN  NaN     NaN      NaN        NaN   
 ...                ...   ...  ...     ...  ...     ...      ...        ...   
 8712  1985-12-30 21:00  91.0  6.0  1014.7 -0.2  1003.8      8.0         Ac   
 8713  1985-12-30 22:00   NaN  NaN     NaN  NaN     NaN      NaN        NaN   
 8714  1985-12-30 23:00   NaN  NaN     NaN  NaN     NaN      NaN        NaN   
 8715  1985-12-31 00:00  79.0  4.5  1017.5 -4.1  1006.5      2.0         Ac   
 8716  1985-12-31 01:00   NaN  NaN     NaN  NaN     NaN      NaN        NaN   
 
       lcsCh  m005Te  m01Te  m02Te

In [25]:
# asos_1985_1996_df = pd.DataFrame([], columns=asos_dataframes['asos_1985_df'])
# for asos_datafram in asos_dataframes:
#     print(asos_datafram)
#     asos_1985_1996_df = pd.concat([asos_1985_1996_df,asos_dataframes[asos_datafram]])
# asos_1985_1996_df

In [26]:
# 위 방법이 너무 오래 걸려 아래방법으로 대치
asos_1985_1996_df = pd.concat([asos_dataframes['asos_1985_df'],asos_dataframes['asos_1986_df'],asos_dataframes['asos_1987_df'],asos_dataframes['asos_1988_df'],asos_dataframes['asos_1989_df'],asos_dataframes['asos_1990_df'],asos_dataframes['asos_1991_df'],asos_dataframes['asos_1992_df'],asos_dataframes['asos_1993_df'],asos_dataframes['asos_1994_df'],asos_dataframes['asos_1995_df'],asos_dataframes['asos_1996_df']])

In [27]:
asos_1985_1996_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100495 entries, 0 to 4579
Data columns (total 17 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   tm          100495 non-null  object 
 1   hm          33496 non-null   float64
 2   pv          33497 non-null   float64
 3   ps          33497 non-null   float64
 4   td          33497 non-null   float64
 5   pa          33497 non-null   float64
 6   dc10Tca     33497 non-null   float64
 7   clfmAbbrCd  22766 non-null   object 
 8   lcsCh       19225 non-null   float64
 9   m005Te      10933 non-null   float64
 10  m01Te       10933 non-null   float64
 11  m02Te       10933 non-null   float64
 12  m03Te       10933 non-null   float64
 13  vs          33497 non-null   float64
 14  ws          100495 non-null  float64
 15  wd          100495 non-null  int64  
 16  rn          10221 non-null   float64
dtypes: float64(14), int64(1), object(2)
memory usage: 13.8+ MB


In [28]:
asos_df = pd.concat([asos_1980_1984_df, asos_1985_1996_df]) # 두 파일간 컬럼이 달라 컬럼 통일후 concat

In [29]:
asos_df.reset_index(drop=True, inplace=True)

In [30]:
asos_df['rainy_day'] = (asos_df['rn'] > 0.0).astype(dtype=int) # 강수량이 0보다 클 경우 1, 0일경우 0

In [31]:
asos_df['rn'] = asos_df['rn'].map(lambda x : 0.0 if pd.isna(x) else x) # 비가 안왔을 경우 NaN -> 강수량을 0.0으로 변환

In [32]:
asos_df = asos_df.apply(lambda x : no_clouds(x), axis=1)

In [33]:
asos_df[(asos_df['lcsCh'] == 0.0)]
asos_df = asos_df.apply(lambda x : no_lcsCh(x), axis=1)
asos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138087 entries, 0 to 138086
Data columns (total 18 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   tm          138087 non-null  object 
 1   hm          46025 non-null   float64
 2   pv          42632 non-null   float64
 3   ps          42628 non-null   float64
 4   td          42632 non-null   float64
 5   pa          42625 non-null   float64
 6   dc10Tca     42632 non-null   float64
 7   clfmAbbrCd  137968 non-null  object 
 8   lcsCh       131712 non-null  float64
 9   m005Te      17198 non-null   float64
 10  m01Te       17198 non-null   float64
 11  m02Te       17198 non-null   float64
 12  m03Te       17198 non-null   float64
 13  vs          42632 non-null   float64
 14  ws          138077 non-null  float64
 15  wd          138087 non-null  int64  
 16  rn          138087 non-null  float64
 17  rainy_day   138087 non-null  int64  
dtypes: float64(14), int64(2), object(2)
memory u

In [34]:
asos_df['lcsCh'] = asos_df['lcsCh'].interpolate(method='linear')

In [35]:
asos_df['clfmAbbrCd'] = asos_df['clfmAbbrCd'].fillna(method='ffill').fillna(method='bfill')

In [36]:
asos_df[['m005Te','m01Te','m02Te','m03Te']] = asos_df[['m005Te','m01Te','m02Te','m03Te']].interpolate(method='linear')

In [37]:
asos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138087 entries, 0 to 138086
Data columns (total 18 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   tm          138087 non-null  object 
 1   hm          46025 non-null   float64
 2   pv          42632 non-null   float64
 3   ps          42628 non-null   float64
 4   td          42632 non-null   float64
 5   pa          42625 non-null   float64
 6   dc10Tca     42632 non-null   float64
 7   clfmAbbrCd  138087 non-null  object 
 8   lcsCh       138087 non-null  float64
 9   m005Te      138085 non-null  float64
 10  m01Te       138085 non-null  float64
 11  m02Te       138085 non-null  float64
 12  m03Te       138085 non-null  float64
 13  vs          42632 non-null   float64
 14  ws          138077 non-null  float64
 15  wd          138087 non-null  int64  
 16  rn          138087 non-null  float64
 17  rainy_day   138087 non-null  int64  
dtypes: float64(14), int64(2), object(2)
memory u

In [38]:
# 나머지 데이터들은 선형 및 시간 보간법을 이용하면 데이터 회손이 크다고 판단되어 드랍
drop_index = asos_df[(asos_df['hm'].isna())|(asos_df['pa'].isna())].index
asos_df.drop(index=drop_index, inplace=True)

In [39]:
asos_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42624 entries, 2 to 138085
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tm          42624 non-null  object 
 1   hm          42624 non-null  float64
 2   pv          42624 non-null  float64
 3   ps          42620 non-null  float64
 4   td          42624 non-null  float64
 5   pa          42624 non-null  float64
 6   dc10Tca     42624 non-null  float64
 7   clfmAbbrCd  42624 non-null  object 
 8   lcsCh       42624 non-null  float64
 9   m005Te      42624 non-null  float64
 10  m01Te       42624 non-null  float64
 11  m02Te       42624 non-null  float64
 12  m03Te       42624 non-null  float64
 13  vs          42624 non-null  float64
 14  ws          42621 non-null  float64
 15  wd          42624 non-null  int64  
 16  rn          42624 non-null  float64
 17  rainy_day   42624 non-null  int64  
dtypes: float64(14), int64(2), object(2)
memory usage: 6.2+ MB


In [40]:
asos_df.to_csv('/Volumes/ESD-ISO/project_preson/project_data/웹크롤링데이터/1980-1996ASOS_관측데이터.csv',index=False)

In [41]:
asos_2020_2024 = pd.read_csv('/Volumes/ESD-ISO/project_preson/shared_data_preparation/ASOS_data_preprocessed_V1.csv')

In [42]:
asos_df = pd.concat([asos_df,asos_2020_2024]).reset_index(drop=True)

In [43]:
asos_df.to_csv('/Volumes/ESD-ISO/project_preson/shared_data_preparation/ASOS_data_preprocessed_V2.csv', index=False)

## 데이터 추가 크롤링(1997-2007) 후 concat과 nan처리 ASOS_data_preprocessed_V3

In [44]:
asos_dataframes = {}
base_path = '/Volumes/ESD-ISO/project_preson/project_data/웹크롤링데이터/'

for year in range(1997, 2008):  # 1997년부터 2007년까지
    file_path = f"{base_path}{year}0101-{year}1231ASOS_관측데이터.csv"
    asos_dataframes[f"asos_{year}_df"] = pd.read_csv(file_path)

In [45]:
asos_1997_2008_df = pd.DataFrame([], columns=asos_dataframes['asos_1997_df'].columns)
for asos_datafram in asos_dataframes:
    asos_1997_2008_df = pd.concat([asos_1980_1984_df,asos_dataframes[asos_datafram]])
asos_1997_2008_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38872 entries, 0 to 1279
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tm          38872 non-null  object 
 1   hm          13809 non-null  float64
 2   pv          9561 non-null   float64
 3   ps          9557 non-null   float64
 4   td          9561 non-null   float64
 5   pa          9554 non-null   float64
 6   dc10Tca     9561 non-null   float64
 7   clfmAbbrCd  6519 non-null   object 
 8   lcsCh       3709 non-null   float64
 9   m005Te      6478 non-null   float64
 10  m01Te       6478 non-null   float64
 11  m02Te       6478 non-null   float64
 12  m03Te       6478 non-null   float64
 13  vs          9561 non-null   float64
 14  ws          38862 non-null  float64
 15  wd          38872 non-null  object 
 16  rn          3561 non-null   float64
dtypes: float64(14), object(3)
memory usage: 5.3+ MB


In [46]:
asos_1997_2008_df['rn'] = asos_1997_2008_df['rn'].map(lambda x : 0.0 if pd.isna(x) else x)

In [47]:
asos_1997_2008_df['rainy_day'] = (asos_1997_2008_df['rn'] > 0.0).astype(dtype=int) # 강수량이 0보다 클 경우 1, 0일경우 0

In [48]:
asos_1997_2008_df = asos_1997_2008_df.apply(lambda x : no_clouds(x), axis=1)

In [49]:
asos_1997_2008_df[(asos_1997_2008_df['lcsCh'] == 0.0)]
asos_1997_2008_df = asos_1997_2008_df.apply(lambda x : no_lcsCh(x), axis=1)
asos_1997_2008_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38872 entries, 0 to 1279
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tm          38872 non-null  object 
 1   hm          13809 non-null  float64
 2   pv          9561 non-null   float64
 3   ps          9557 non-null   float64
 4   td          9561 non-null   float64
 5   pa          9554 non-null   float64
 6   dc10Tca     9561 non-null   float64
 7   clfmAbbrCd  38845 non-null  object 
 8   lcsCh       36035 non-null  float64
 9   m005Te      6478 non-null   float64
 10  m01Te       6478 non-null   float64
 11  m02Te       6478 non-null   float64
 12  m03Te       6478 non-null   float64
 13  vs          9561 non-null   float64
 14  ws          38862 non-null  float64
 15  wd          38872 non-null  int64  
 16  rn          38872 non-null  float64
 17  rainy_day   38872 non-null  int64  
dtypes: float64(14), int64(2), object(2)
memory usage: 5.6+ MB


In [50]:
asos_1997_2008_df['lcsCh'] = asos_1997_2008_df['lcsCh'].interpolate(method='linear')

In [51]:
asos_1997_2008_df['clfmAbbrCd'] = asos_1997_2008_df['clfmAbbrCd'].fillna(method='ffill').fillna(method='bfill')

In [52]:
asos_1997_2008_df[['m005Te','m01Te','m02Te','m03Te']] = asos_1997_2008_df[['m005Te','m01Te','m02Te','m03Te']].interpolate(method='linear')

In [53]:
asos_1997_2008_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38872 entries, 0 to 1279
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tm          38872 non-null  object 
 1   hm          13809 non-null  float64
 2   pv          9561 non-null   float64
 3   ps          9557 non-null   float64
 4   td          9561 non-null   float64
 5   pa          9554 non-null   float64
 6   dc10Tca     9561 non-null   float64
 7   clfmAbbrCd  38872 non-null  object 
 8   lcsCh       38872 non-null  float64
 9   m005Te      38870 non-null  float64
 10  m01Te       38870 non-null  float64
 11  m02Te       38870 non-null  float64
 12  m03Te       38870 non-null  float64
 13  vs          9561 non-null   float64
 14  ws          38862 non-null  float64
 15  wd          38872 non-null  int64  
 16  rn          38872 non-null  float64
 17  rainy_day   38872 non-null  int64  
dtypes: float64(14), int64(2), object(2)
memory usage: 5.6+ MB


In [54]:
asos_1997_2008_df.reset_index(drop=True, inplace=True)

In [55]:
asos_1997_2008_df[(asos_1997_2008_df['ps'].isna())|(asos_1997_2008_df['pa'].isna())].index

Int64Index([    0,     1,     3,     4,     5,     6,     7,     9,    10,
               11,
            ...
            38856, 38857, 38859, 38860, 38863, 38864, 38866, 38867, 38869,
            38870],
           dtype='int64', length=29322)

In [56]:
drop_index = asos_1997_2008_df[(asos_1997_2008_df['ps'].isna())|(asos_1997_2008_df['pa'].isna())].index
asos_1997_2008_df.drop(index=drop_index, inplace=True)

In [57]:
asos_1997_2008_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9550 entries, 2 to 38871
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tm          9550 non-null   object 
 1   hm          9550 non-null   float64
 2   pv          9550 non-null   float64
 3   ps          9550 non-null   float64
 4   td          9550 non-null   float64
 5   pa          9550 non-null   float64
 6   dc10Tca     9550 non-null   float64
 7   clfmAbbrCd  9550 non-null   object 
 8   lcsCh       9550 non-null   float64
 9   m005Te      9550 non-null   float64
 10  m01Te       9550 non-null   float64
 11  m02Te       9550 non-null   float64
 12  m03Te       9550 non-null   float64
 13  vs          9550 non-null   float64
 14  ws          9547 non-null   float64
 15  wd          9550 non-null   int64  
 16  rn          9550 non-null   float64
 17  rainy_day   9550 non-null   int64  
dtypes: float64(14), int64(2), object(2)
memory usage: 1.4+ MB


In [58]:
asos_1997_2008_df.to_csv('/Volumes/ESD-ISO/project_preson/project_data/웹크롤링데이터/1997-2008ASOS_관측데이터.csv', index=False)

In [59]:
asos_1980_1997_df = pd.read_csv('/Volumes/ESD-ISO/project_preson/project_data/웹크롤링데이터/1980-1996ASOS_관측데이터.csv')

In [60]:
asos_2020_2024_df = pd.read_csv('/Volumes/ESD-ISO/project_preson/LSTM/LSTM_data/ASOS_data_preprocessed_V1.csv')

In [61]:
asos_df = pd.concat([asos_1980_1997_df,asos_1997_2008_df,asos_2020_2024_df]).reset_index(drop=True)

In [62]:
asos_df.to_csv('/Volumes/ESD-ISO/project_preson/shared_data_preparation/ASOS_data_preprocessed_V3.csv', index=False)

## 데이터 추가 크롤링(1997-2007) 후 concat과 nan처리 ASOS_data_preprocessed_V4

In [150]:
asos_1980_1996_df = pd.read_csv('/Volumes/ESD-ISO/project_preson/project_data/웹크롤링데이터/1980-1996ASOS_관측데이터.csv')

In [151]:
asos_1997_2008_df = pd.read_csv('/Volumes/ESD-ISO/project_preson/project_data/웹크롤링데이터/1997-2008ASOS_관측데이터.csv')

In [152]:
asos_dataframes = {}
base_path = '/Volumes/ESD-ISO/project_preson/project_data/웹크롤링데이터/'

for year in range(2009, 2020):
    file_path = f"{base_path}{year}0101-{year}1231ASOS_관측데이터.csv"
    asos_dataframes[f"asos_{year}_df"] = pd.read_csv(file_path)

In [154]:
asos_2009_2019_df = pd.DataFrame([], columns=asos_dataframes['asos_2009_df'].columns)
for asos_datafram in asos_dataframes:
    asos_2009_2019_df = pd.concat([asos_1980_1984_df,asos_dataframes[asos_datafram]])
asos_2009_2019_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46149 entries, 0 to 8556
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tm          46149 non-null  object 
 1   hm          21086 non-null  float64
 2   pv          17692 non-null  float64
 3   ps          17688 non-null  float64
 4   td          17692 non-null  float64
 5   pa          17685 non-null  float64
 6   dc10Tca     17682 non-null  float64
 7   clfmAbbrCd  11056 non-null  object 
 8   lcsCh       7783 non-null   float64
 9   m005Te      14822 non-null  float64
 10  m01Te       14822 non-null  float64
 11  m02Te       14822 non-null  float64
 12  m03Te       14822 non-null  float64
 13  vs          17692 non-null  float64
 14  ws          46132 non-null  float64
 15  wd          46142 non-null  object 
 16  rn          4339 non-null   float64
dtypes: float64(14), object(3)
memory usage: 6.3+ MB


In [155]:
asos_2020_2024_df = pd.read_csv('/Volumes/ESD-ISO/project_preson/LSTM/LSTM_data/ASOS_data_preprocessed_V1.csv')

In [156]:
asos_2009_2019_df['rn'] = asos_1997_2008_df['rn'].map(lambda x : 0.0 if pd.isna(x) else x)

In [157]:
asos_2009_2019_df['rainy_day'] = (asos_2009_2019_df['rn'] > 0.0).astype(dtype=int) # 강수량이 0보다 클 경우 1, 0일경우 0

In [158]:
asos_2009_2019_df = asos_2009_2019_df.apply(lambda x : no_clouds(x), axis=1)

In [159]:
asos_2009_2019_df[(asos_2009_2019_df['lcsCh'] == 0.0)]
asos_2009_2019_df = asos_2009_2019_df.apply(lambda x : no_lcsCh(x), axis=1)
asos_2009_2019_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46149 entries, 0 to 8556
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tm          46149 non-null  object 
 1   hm          21086 non-null  float64
 2   pv          17692 non-null  float64
 3   ps          17688 non-null  float64
 4   td          17692 non-null  float64
 5   pa          17685 non-null  float64
 6   dc10Tca     17682 non-null  float64
 7   clfmAbbrCd  45324 non-null  object 
 8   lcsCh       42051 non-null  float64
 9   m005Te      14822 non-null  float64
 10  m01Te       14822 non-null  float64
 11  m02Te       14822 non-null  float64
 12  m03Te       14822 non-null  float64
 13  vs          17692 non-null  float64
 14  ws          46132 non-null  float64
 15  wd          46142 non-null  float64
 16  rn          46149 non-null  float64
 17  rainy_day   46149 non-null  int64  
dtypes: float64(15), int64(1), object(2)
memory usage: 6.7+ MB


In [160]:
asos_2009_2019_df['lcsCh'] = asos_2009_2019_df['lcsCh'].interpolate(method='linear')
asos_2009_2019_df['clfmAbbrCd'] = asos_2009_2019_df['clfmAbbrCd'].fillna(method='ffill').fillna(method='bfill')
asos_2009_2019_df[['m005Te','m01Te','m02Te','m03Te']] = asos_2009_2019_df[['m005Te','m01Te','m02Te','m03Te']].interpolate(method='linear')
asos_2009_2019_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46149 entries, 0 to 8556
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tm          46149 non-null  object 
 1   hm          21086 non-null  float64
 2   pv          17692 non-null  float64
 3   ps          17688 non-null  float64
 4   td          17692 non-null  float64
 5   pa          17685 non-null  float64
 6   dc10Tca     17682 non-null  float64
 7   clfmAbbrCd  46149 non-null  object 
 8   lcsCh       46149 non-null  float64
 9   m005Te      46147 non-null  float64
 10  m01Te       46147 non-null  float64
 11  m02Te       46147 non-null  float64
 12  m03Te       46147 non-null  float64
 13  vs          17692 non-null  float64
 14  ws          46132 non-null  float64
 15  wd          46142 non-null  float64
 16  rn          46149 non-null  float64
 17  rainy_day   46149 non-null  int64  
dtypes: float64(15), int64(1), object(2)
memory usage: 6.7+ MB


In [161]:
asos_2009_2019_df.reset_index(drop=True, inplace=True)

In [163]:
drop_index = asos_2009_2019_df[(asos_2009_2019_df['ps'].isna())|(asos_2009_2019_df['pa'].isna())].index
asos_2009_2019_df.drop(index=drop_index, inplace=True)

In [165]:
asos_df = pd.concat([asos_1980_1996_df,asos_1997_2008_df,asos_2009_2019_df,asos_2020_2024_df]).reset_index(drop=True)

In [166]:
asos_df.to_csv('/Volumes/ESD-ISO/project_preson/shared_data_preparation/ASOS_data_preprocessed_V4.csv', index=False)

In [167]:
asos_2009_2019_df.to_csv('/Volumes/ESD-ISO/project_preson/project_data/웹크롤링데이터/2009-2019ASOS_관측데이터.csv')

## 데이터 추가 크롤링(2000-2019) 후 concat과 nan처리 컬럼 drop버전

In [106]:
asos_dataframes = {}
base_path = '/Volumes/ESD-ISO/project_preson/project_data/웹크롤링데이터/'

for year in range(1980, 1985):  # 1980년부터 1984년까지
    file_path = f"{base_path}{year}0101-{year}1231ASOS_관측데이터.csv"
    asos_dataframes[f"asos_{year}_df"] = pd.read_csv(file_path)

In [107]:
asos_1980_1984_df = pd.DataFrame([], columns=asos_dataframes['asos_1980_df'].columns)
for asos_datafram in asos_dataframes:
    asos_1980_1984_df = pd.concat([asos_1980_1984_df,asos_dataframes[asos_datafram]])
asos_1980_1984_df.columns = ['tm', 'hm', 'pv', 'ps', 'td', 'pa', 'dc10Tca', 'clfmAbbrCd', 'lcsCh', 'm005Te', 'm01Te', 'm02Te', 'm03Te', 'vs', 'ws', 'wd', 'rn']
asos_1980_1984_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37592 entries, 0 to 2649
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tm          37592 non-null  object 
 1   hm          12529 non-null  float64
 2   pv          9135 non-null   float64
 3   ps          9131 non-null   float64
 4   td          9135 non-null   float64
 5   pa          9128 non-null   float64
 6   dc10Tca     9135 non-null   float64
 7   clfmAbbrCd  6271 non-null   object 
 8   lcsCh       3556 non-null   float64
 9   m005Te      6265 non-null   float64
 10  m01Te       6265 non-null   float64
 11  m02Te       6265 non-null   float64
 12  m03Te       6265 non-null   float64
 13  vs          9135 non-null   float64
 14  ws          37582 non-null  float64
 15  wd          37592 non-null  object 
 16  rn          3531 non-null   float64
dtypes: float64(14), object(3)
memory usage: 5.2+ MB


In [108]:
asos_dataframes = {}
base_path = '/Volumes/ESD-ISO/project_preson/project_data/웹크롤링데이터/'

for year in range(1985, 2020):  # 1985년부터 2019년까지
    file_path = f"{base_path}{year}0101-{year}1231ASOS_관측데이터.csv"
    asos_dataframes[f"asos_{year}_df"] = pd.read_csv(file_path)

In [109]:
asos_1985_2019_df = pd.DataFrame([], columns=asos_dataframes['asos_1990_df'].columns)
for asos_datafram in asos_dataframes:
    asos_1985_2019_df = pd.concat([asos_1980_1984_df,asos_dataframes[asos_datafram]])
asos_1985_2019_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46149 entries, 0 to 8556
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tm          46149 non-null  object 
 1   hm          21086 non-null  float64
 2   pv          17692 non-null  float64
 3   ps          17688 non-null  float64
 4   td          17692 non-null  float64
 5   pa          17685 non-null  float64
 6   dc10Tca     17682 non-null  float64
 7   clfmAbbrCd  11056 non-null  object 
 8   lcsCh       7783 non-null   float64
 9   m005Te      14822 non-null  float64
 10  m01Te       14822 non-null  float64
 11  m02Te       14822 non-null  float64
 12  m03Te       14822 non-null  float64
 13  vs          17692 non-null  float64
 14  ws          46132 non-null  float64
 15  wd          46142 non-null  object 
 16  rn          4339 non-null   float64
dtypes: float64(14), object(3)
memory usage: 6.3+ MB


In [110]:
path = '/Volumes/ESD-ISO/project_preson/project_data/웹크롤링데이터/2020-2024ASOS_관측데이터.csv'
asos_2020_2024_df = pd.read_csv(path)
asos_2020_2024_df = asos_2020_2024_df.iloc[:, 0].apply(ast.literal_eval)  # 문자열을 딕셔너리로 변환
asos_2020_2024_df = pd.json_normalize(asos_2020_2024_df)  # 딕셔너리를 열로

In [111]:
asos_2020_2024_df['rn'] = asos_2020_2024_df['rn'].map(lambda x :  float(x.split()[0]) if x else 0.0) # 이 년도만 rn이 object 타입

In [112]:
asos_2020_2024_df['clfmAbbrCd'] = asos_2020_2024_df['clfmAbbrCd'].map(lambda x : np.nan if x == '' else x)

In [113]:
asos_df = pd.concat([asos_1980_1984_df, asos_1985_2019_df, asos_2020_2024_df])

In [114]:
asos_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127394 entries, 0 to 43652
Data columns (total 17 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   tm          127394 non-null  object 
 1   hm          77268 non-null   object 
 2   pv          70480 non-null   object 
 3   ps          70472 non-null   object 
 4   td          70480 non-null   object 
 5   pa          70466 non-null   object 
 6   dc10Tca     70470 non-null   object 
 7   clfmAbbrCd  41589 non-null   object 
 8   lcsCh       54992 non-null   object 
 9   m005Te      64740 non-null   object 
 10  m01Te       64740 non-null   object 
 11  m02Te       64740 non-null   object 
 12  m03Te       64740 non-null   object 
 13  vs          70480 non-null   object 
 14  ws          127367 non-null  object 
 15  wd          127387 non-null  object 
 16  rn          51523 non-null   float64
dtypes: float64(1), object(16)
memory usage: 17.5+ MB


In [115]:
# 사용할 컬럼 ['tm','hm','pv','td','pa','m005Te','m01Te','m02Te','m03Te','clfmAbbrCd','lcsCh']

In [116]:
asos_df['rn'] = asos_df['rn'].fillna(0.0)

In [117]:
# 숫자로 변환할 수 없는 값을 NaN으로 처리
asos_df[['hm', 'pv', 'td', 'pa', 'm005Te', 'm01Te', 'm02Te', 'm03Te', 'lcsCh']] = asos_df[
    ['hm', 'pv', 'td', 'pa', 'm005Te', 'm01Te', 'm02Te', 'm03Te', 'lcsCh']
].apply(pd.to_numeric, errors='coerce')

In [118]:
asos_df[['hm','pv','td','pa','m005Te','m01Te','m02Te','m03Te','lcsCh']].astype('float')

,hm,pv,td,pa,m005Te,m01Te,m02Te,m03Te,lcsCh
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,56.0,3.9,-6.0,1015.9,-1.2,0.3,0.5,0.6,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
43648,67.0,6.1,0.0,1005.4,-0.1,-0.3,0.8,1.7,NaN
43649,71.0,6.1,0.0,1005.6,-0.1,-0.3,0.8,1.7,NaN
43650,75.0,6.1,0.0,1005.4,-0.1,-0.3,0.8,1.7,NaN
43651,81.0,7.0,1.8,1005.2,-0.1,-0.3,0.8,1.7,NaN


In [119]:
asos_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127394 entries, 0 to 43652
Data columns (total 17 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   tm          127394 non-null  object 
 1   hm          77268 non-null   float64
 2   pv          70480 non-null   float64
 3   ps          70472 non-null   object 
 4   td          70480 non-null   float64
 5   pa          70456 non-null   float64
 6   dc10Tca     70470 non-null   object 
 7   clfmAbbrCd  41589 non-null   object 
 8   lcsCh       33980 non-null   float64
 9   m005Te      64726 non-null   float64
 10  m01Te       64726 non-null   float64
 11  m02Te       64726 non-null   float64
 12  m03Te       64726 non-null   float64
 13  vs          70480 non-null   object 
 14  ws          127367 non-null  object 
 15  wd          127387 non-null  object 
 16  rn          127394 non-null  float64
dtypes: float64(10), object(7)
memory usage: 17.5+ MB


In [120]:
def no_lcsCh(row):
    if (pd.isna(row['lcsCh'])) & (row['clfmAbbrCd'] == 'no_clouds'):
        row['lcsCh'] = 0.0
        return row
    else:
        return row

In [121]:
def no_clouds(row):
    if (pd.isna(row['lcsCh'])) & (pd.isna(row['clfmAbbrCd'])):
        row['clfmAbbrCd'] = 'no_clouds'
        return row
    else:
        return row

In [122]:
asos_df = asos_df.apply(lambda x : no_clouds(x), axis=1)

In [123]:
asos_df = asos_df.apply(lambda x : no_lcsCh(x), axis=1)

In [124]:
asos_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127394 entries, 0 to 43652
Data columns (total 17 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   tm          127394 non-null  object 
 1   hm          77268 non-null   float64
 2   pv          70480 non-null   float64
 3   ps          70472 non-null   object 
 4   td          70480 non-null   float64
 5   pa          70456 non-null   float64
 6   dc10Tca     70470 non-null   object 
 7   clfmAbbrCd  122741 non-null  object 
 8   lcsCh       115132 non-null  float64
 9   m005Te      64726 non-null   float64
 10  m01Te       64726 non-null   float64
 11  m02Te       64726 non-null   float64
 12  m03Te       64726 non-null   float64
 13  vs          70480 non-null   object 
 14  ws          127367 non-null  object 
 15  wd          127387 non-null  object 
 16  rn          127394 non-null  float64
dtypes: float64(10), object(7)
memory usage: 17.5+ MB


In [125]:
# 운형이 있지만 최저운고가 없는경우 선형보간법으로 대치
asos_df['lcsCh'] = asos_df['lcsCh'].interpolate(method='linear')

In [126]:
# 최저운고가 있지만 운형이 없는경우 시간적 보간법으로 대치
asos_df['clfmAbbrCd'] = asos_df['clfmAbbrCd'].fillna(method='ffill').fillna(method='bfill')

In [127]:
asos_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127394 entries, 0 to 43652
Data columns (total 17 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   tm          127394 non-null  object 
 1   hm          77268 non-null   float64
 2   pv          70480 non-null   float64
 3   ps          70472 non-null   object 
 4   td          70480 non-null   float64
 5   pa          70456 non-null   float64
 6   dc10Tca     70470 non-null   object 
 7   clfmAbbrCd  127394 non-null  object 
 8   lcsCh       127394 non-null  float64
 9   m005Te      64726 non-null   float64
 10  m01Te       64726 non-null   float64
 11  m02Te       64726 non-null   float64
 12  m03Te       64726 non-null   float64
 13  vs          70480 non-null   object 
 14  ws          127367 non-null  object 
 15  wd          127387 non-null  object 
 16  rn          127394 non-null  float64
dtypes: float64(10), object(7)
memory usage: 17.5+ MB


In [128]:
# 사용할 컬럼중 수치형으로 다음 날씨까지 선형적으로 이어질것같은 온,습도의 경우 선형보간법
asos_df[['m005Te','m01Te','m02Te','m03Te', 'td']] = asos_df[['m005Te','m01Te','m02Te','m03Te', 'td']].interpolate(method='linear')

In [129]:
asos_df['hm'] = asos_df['hm'].interpolate(method='linear')

In [130]:
asos_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127394 entries, 0 to 43652
Data columns (total 17 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   tm          127394 non-null  object 
 1   hm          127392 non-null  float64
 2   pv          70480 non-null   float64
 3   ps          70472 non-null   object 
 4   td          127392 non-null  float64
 5   pa          70456 non-null   float64
 6   dc10Tca     70470 non-null   object 
 7   clfmAbbrCd  127394 non-null  object 
 8   lcsCh       127394 non-null  float64
 9   m005Te      127392 non-null  float64
 10  m01Te       127392 non-null  float64
 11  m02Te       127392 non-null  float64
 12  m03Te       127392 non-null  float64
 13  vs          70480 non-null   object 
 14  ws          127367 non-null  object 
 15  wd          127387 non-null  object 
 16  rn          127394 non-null  float64
dtypes: float64(10), object(7)
memory usage: 17.5+ MB


In [131]:
# 기압은 계절 및 기후의 영향을 많이 받기 때문에 계절, 비의 여부의 따라 평균치로 대치

In [132]:
asos_df['rain'] = (asos_df['rn'] > 0.0).astype(dtype=int) # 비가 오는지 안오는지에 대한 여부컬럼

In [134]:
asos_df['tm'] = asos_df['tm'].map(lambda x : x+':00' if len(x) != 19 else x) # tm컬럼의 시간이 년도별로 시간:분 or 시간:분:초로 되어있어서 변환
asos_df['tm'] = pd.to_datetime(asos_df['tm'])
asos_df['ps'] = asos_df['ps'].map(lambda x : np.nan if x == '' else x)
asos_df['ps'] = asos_df['ps'].astype('float')

# 계절별 데이터 필터링 함수 정의
def extract_season_data(data, season):
    if season == 'spring':
        return data[(data['tm'].dt.month >= 3) & (data['tm'].dt.month <= 5)]
    elif season == 'summer':
        return data[(data['tm'].dt.month >= 6) & (data['tm'].dt.month <= 8)]
    elif season == 'fall':
        return data[(data['tm'].dt.month >= 9) & (data['tm'].dt.month <= 11)]
    elif season == 'winter':
        return data[(data['tm'].dt.month == 12) | (data['tm'].dt.month <= 2)]
    else:
        return None

# 계절별 데이터 추출
spring_data = extract_season_data(asos_df, 'spring')
summer_data = extract_season_data(asos_df, 'summer')
fall_data = extract_season_data(asos_df, 'fall')
winter_data = extract_season_data(asos_df, 'winter')

In [135]:
spring_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33244 entries, 1439 to 38538
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   tm          33244 non-null  datetime64[ns]
 1   hm          33244 non-null  float64       
 2   pv          17904 non-null  float64       
 3   ps          17904 non-null  float64       
 4   td          33244 non-null  float64       
 5   pa          17898 non-null  float64       
 6   dc10Tca     17898 non-null  object        
 7   clfmAbbrCd  33244 non-null  object        
 8   lcsCh       33244 non-null  float64       
 9   m005Te      33244 non-null  float64       
 10  m01Te       33244 non-null  float64       
 11  m02Te       33244 non-null  float64       
 12  m03Te       33244 non-null  float64       
 13  vs          17904 non-null  object        
 14  ws          33244 non-null  object        
 15  wd          33244 non-null  object        
 16  rn          33244 n

In [136]:
spring_mean_data = spring_data.groupby(['rain'])[['pa','ps','pv']].mean()

In [137]:
summer_mean_data = summer_data.groupby(['rain'])[['pa','ps','pv']].mean()

In [138]:
fall_mean_data = fall_data.groupby(['rain'])[['pa','ps','pv']].mean()

In [139]:
winter_mean_data = winter_data.groupby(['rain'])[['pa','ps','pv']].mean()

In [140]:
spring_mean_data

,pa,ps,pv
rain,,,
0,1005.304396,1015.602308,8.603363
1,998.748148,1009.022399,13.600970


In [141]:
def season_pressure(data):
    
    if pd.isna(data['pa']):
        if (data['tm'].month >= 3) & (data['tm'].month <= 5):
            if data['rain'] == 1:
                data['pa'] = spring_mean_data.loc[1,'pa']
            else:
                data['pa'] = spring_mean_data.loc[0,'pa']
        elif (data['tm'].month >= 6) & (data['tm'].month <= 8):
            if data['rain'] == 1:
                data['pa'] = summer_mean_data.loc[1,'pa']
            else:
                data['pa'] = summer_mean_data.loc[0,'pa']
        elif (data['tm'].month >= 9) & (data['tm'].month <= 11):
            if data['rain'] == 1:
                data['pa'] = fall_mean_data.loc[1,'pa']
            else:
                data['pa'] = fall_mean_data.loc[0,'pa']
        elif (data['tm'].month == 12) | (data['tm'].month <= 2):
            if data['rain'] == 1:
                data['pa'] = winter_mean_data.loc[1,'pa']
            else:
                data['pa'] = winter_mean_data.loc[0,'pa']
                
    if pd.isna(data['ps']):
        if (data['tm'].month >= 3) & (data['tm'].month <= 5):
            if data['rain'] == 1:
                data['ps'] = spring_mean_data.loc[1,'ps']
            else:
                data['ps'] = spring_mean_data.loc[0,'ps']
        elif (data['tm'].month >= 6) & (data['tm'].month <= 8):
            if data['rain'] == 1:
                data['ps'] = summer_mean_data.loc[1,'ps']
            else:
                data['ps'] = summer_mean_data.loc[0,'ps']
        elif (data['tm'].month >= 9) & (data['tm'].month <= 11):
            if data['rain'] == 1:
                data['ps'] = fall_mean_data.loc[1,'ps']
            else:
                data['ps'] = fall_mean_data.loc[0,'ps']
        elif (data['tm'].month == 12) | (data['tm'].month <= 2):
            if data['rain'] == 1:
                data['ps'] = winter_mean_data.loc[1,'ps']
            else:
                data['ps'] = winter_mean_data.loc[0,'ps']
    
    if pd.isna(data['pv']):
        if (data['tm'].month >= 3) & (data['tm'].month <= 5):
            if data['rain'] == 1:
                data['pv'] = spring_mean_data.loc[1,'pv']
            else:
                data['pv'] = spring_mean_data.loc[0,'pv']
        elif (data['tm'].month >= 6) & (data['tm'].month <= 8):
            if data['rain'] == 1:
                data['pv'] = summer_mean_data.loc[1,'pv']
            else:
                data['pv'] = summer_mean_data.loc[0,'pv']
        elif (data['tm'].month >= 9) & (data['tm'].month <= 11):
            if data['rain'] == 1:
                data['pv'] = fall_mean_data.loc[1,'pv']
            else:
                data['pv'] = fall_mean_data.loc[0,'pv']
        elif (data['tm'].month == 12) | (data['tm'].month <= 2):
            if data['rain'] == 1:
                data['pv'] = winter_mean_data.loc[1,'pv']
            else:
                data['pv'] = winter_mean_data.loc[0,'pv']
    return data

In [142]:
asos_df = asos_df.apply(lambda x : season_pressure(x), axis=1)

In [143]:
asos_df.reset_index(drop=True, inplace=True)

In [144]:
asos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127394 entries, 0 to 127393
Data columns (total 18 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   tm          127394 non-null  datetime64[ns]
 1   hm          127392 non-null  float64       
 2   pv          127394 non-null  float64       
 3   ps          127394 non-null  float64       
 4   td          127392 non-null  float64       
 5   pa          127394 non-null  float64       
 6   dc10Tca     70470 non-null   object        
 7   clfmAbbrCd  127394 non-null  object        
 8   lcsCh       127394 non-null  float64       
 9   m005Te      127392 non-null  float64       
 10  m01Te       127392 non-null  float64       
 11  m02Te       127392 non-null  float64       
 12  m03Te       127392 non-null  float64       
 13  vs          70480 non-null   object        
 14  ws          127367 non-null  object        
 15  wd          127387 non-null  object        
 16  rn

In [145]:
asos_df.to_csv('/Volumes/ESD-ISO/project_preson/shared_data_preparation/ASOS_1980_2024_Before_drop.csv', index=False)

In [146]:
# 이제 사용하지 않은 컬럼 삭제
# 사용할 컬럼 ['tm','hm','pv','td','pa','m005Te','m01Te','m02Te','m03Te','clfmAbbrCd','lcsCh']

In [147]:
asos_df.drop(columns=['dc10Tca','ps','vs','ws','wd','rn'],inplace=True)

In [148]:
asos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127394 entries, 0 to 127393
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   tm          127394 non-null  datetime64[ns]
 1   hm          127392 non-null  float64       
 2   pv          127394 non-null  float64       
 3   td          127392 non-null  float64       
 4   pa          127394 non-null  float64       
 5   clfmAbbrCd  127394 non-null  object        
 6   lcsCh       127394 non-null  float64       
 7   m005Te      127392 non-null  float64       
 8   m01Te       127392 non-null  float64       
 9   m02Te       127392 non-null  float64       
 10  m03Te       127392 non-null  float64       
 11  rain        127394 non-null  int64         
dtypes: datetime64[ns](1), float64(9), int64(1), object(1)
memory usage: 11.7+ MB


In [149]:
asos_df.to_csv('/Volumes/ESD-ISO/project_preson/shared_data_preparation/ASOS_1980_2024_After_drop.csv', index=False)